In [1]:
import os
import sys
import glob
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon

# 手动处理 DLL 目录
if sys.version_info < (3, 8):
    for p in os.environ["PATH"].split(os.pathsep):
        if glob.glob(os.path.join(p, "gdal*.dll")):
            os.environ['PATH'] = p + os.pathsep + os.environ['PATH']

def get_box_corners(polygon):
    """
    获取一个多边形（矩形）的四个角点坐标。
    """
    if not isinstance(polygon, Polygon):
        raise ValueError("输入的几何对象不是多边形")

    exterior_coords = polygon.exterior.coords[:4]  # 获取多边形的外部坐标，前四个点是角点
    return exterior_coords

def main(shp_file, output_excel):
    # 读取SHP文件
    gdf = gpd.read_file(shp_file)

    # 如果需要，将 GeoDataFrame 转换为 WGS84 地理坐标系
    if gdf.crs != "EPSG:4326":
        gdf = gdf.to_crs(epsg=4326)

    # 存储角点坐标
    corners_list = []

    for idx, row in gdf.iterrows():
        geom = row.geometry
        if geom is None or geom.is_empty or not isinstance(geom, Polygon):
            continue

        corners = get_box_corners(geom)
        corners_list.append({
            "index": idx,
            "corner_1": corners[0],
            "corner_2": corners[1],
            "corner_3": corners[2],
            "corner_4": corners[3]
        })

    # 将结果转换为 DataFrame
    df = pd.DataFrame(corners_list)

    # 保存到 Excel 文件
    df.to_excel(output_excel, index=False)
    print(f"坐标已保存到 {output_excel}")

# 示例文件路径
shp_file = 'your_shp_path.shp'
output_excel = 'output_excel_path.xlsx'

# 执行主函数
main(shp_file, output_excel)

坐标已保存到 C:\Users\jack_\Desktop\shp2.xlsx
